In [ ]:
from common import *
use_hvplot()

In [ ]:
year = 2018
water_year = f'wy{year}'

snotel_sites_lola = SnotelLocations()
snotel_sites_lola.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites-lola.json')

SAVE_DEST = "wy{year}/{site.name}/{site.name}-HRRR-wy{year}_sfc06.zarr"

### Extract data from HRRR grib files

In [ ]:
HRRR_FC = 'wrfsfcf01.apcp06.grib2'

def snotel_hrrr_data(site, hrrr_ref, year):
    hrrr_y, hrrr_x = hrrr_pixel_index(hrrr_ref, site)
    hrrr_parts = []
    print(f'Processing site: {site}')

    for chunk in [f'{year - 1}1*', f'{year}0[1-2]*', f'{year}0[3-4]*', f'{year}0[5-6]*', f'{year}0[7-8]*', f'{year}09*']:
        hrrr_files = sorted((HRRR_DIR).rglob(f'{year}/hrrr.{chunk}/hrrr.t*z.{HRRR_FC}'))
        if not hrrr_files:
            continue
        print(f'  Getting chunk: {chunk}')
        hours = [
            hrrr_snotel_pixel(file, hrrr_x, hrrr_y)
            for file in hrrr_files
        ]
        part = dask.compute(hours)
        hrrr_parts.append(xr.concat(part[0], dim='time'))

    return xr.concat(hrrr_parts, dim='time').resample(**RESAMPLE_1_DAY_OPTS).sum()

In [ ]:
# Use the first file of the water year to determine the HRRR pixel location for a site
hrrr_ref = xr.open_dataset(
    (HRRR_DIR / str(year) / f'hrrr.{year-1}1001/hrrr.t00z.{HRRR_FC}').as_posix(),
    engine='cfgrib',
    backend_kwargs={
        'errors': 'ignore',
        'indexpath': '',
        'filter_by_keys': {
            'level': 0,
            'typeOfLevel': 'surface',
        }
    },
)

In [ ]:
client = start_cluster(5, 12)
client

In [ ]:
schofield_hrrr = snotel_hrrr_data(snotel_sites_lola.Schofield, hrrr_ref, year)
schofield_hrrr.to_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites_lola.Schofield)
)

In [ ]:
butte_hrrr = snotel_hrrr_data(snotel_sites_lola.Butte, hrrr_ref, year)
butte_hrrr.to_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites_lola.Butte)
)

In [ ]:
taylor_hrrr = snotel_hrrr_data(snotel_sites_lola.Taylor, hrrr_ref, year)
taylor_hrrr.to_zarr(
    SNOTEL_DIR / SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites_lola.Taylor)
)

In [ ]:
irwin_hrrr = snotel_hrrr_data(snotel_sites_lola.Irwin, hrrr_ref, year)
irwin_hrrr.to_zarr(
    SNOTEL_DIR / SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites_lola.Irwin)
)

### Verify output

In [ ]:
schofield_hrrr = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites_lola.Schofield)
)

butte_hrrr = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites_lola.Butte)
)

taylor_hrrr = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites_lola.Taylor)
)

irwin_hrrr = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites_lola.Irwin)
)

In [ ]:
schofield_hrrr.tp.mean(['x', 'y']).hvplot(
    line_dash='dashed', label=snotel_sites_lola.Schofield.name,
    height=640, width=1280, title='HRRR FC06 precipitation'
) * \
butte_hrrr.tp.mean(['x', 'y']).hvplot(
    line_dash='dashed', label=snotel_sites_lola.Butte.name
) * \
taylor_hrrr.tp.mean(['x', 'y']).hvplot(
    line_dash='dashed', label=snotel_sites_lola.Taylor.name
) * \
irwin_hrrr.tp.mean(['x', 'y']).hvplot(
    line_dash='dashed', label=snotel_sites_lola.Irwin.name
)